# SESION DE TRABAJO 5. Descripción Gráfica: la revancha (clase práctica)

**¡Bienvenides mis querides programadores!** 

Este Notebook está pensado para ejecutar una serie de códigos que nos ayudará a repasar y aterrizar lo que hasta este punto hemos visto del análisis de datos, haciendo un hincapié en la **ESTADÍSTICA DESCRIPTIVA**. Bueno, como tal haremos lo de siempre, te explico y luego te corresponderá a ti hacer el resto de análisis correspondientes. Te recomiendo que este notebook lo llenes de notas y comentarios y te lo envíes por correo para guardar todo lo aprendido durante las clases. 

Para hacerlo más interesante lo haremos con datos reales. Y claro, con algo relacionado con ciencias de la tierra. No te preocupes, te ayudaré en la primera interpretación, aunque sería bueno que con los conocimientos básicos puedas entender lo que significa lo que estamos haciendo. Así que como verás te presento los datos con los que estaremos trabajando. 

## 0. Introducción de base de datos iBTRAcS
### ¿Qué es IBTrACS?


Trabajaremos con los datos de IBTrACS (International Best Track Archive for Climate Stewardship), una base de datos global que recopila información sobre ciclones tropicales. Este recurso ya se ha mencionado previamente en este curso. Los datos originales se pueden consultar aquí:

Puedes obtener más información [aquí](https://www.ncei.noaa.gov/products/international-best-track-archive).

Como ocurre con muchos proyectos de gran escala, los datos de IBTrACS se encuentran distribuidos en múltiples archivos y formatos. Los datos en su forma original están disponibles en varios formatos como NetCDF y CSV. Aunque pandas puede leer archivos CSV sin problemas, trabajar con formatos como NetCDF requiere un conocimiento más avanzado. Por esta razón, en este curso nos enfocaremos en archivos CSV, que son más comunes y fáciles de manejar.

---

Su profesor ha usado estos datos múltiples veces así para que te des un aire de lo que contiene te dejo un gráfico realizado por mí: 

![](images/fig_1.jpeg)

Los puntitos que ves son posiciones de los ciclones tropicales, y trabajaras con algo similar. Así que manos a la obra...

## 1. Bibliotecas 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Carga de la base de Datos

Pandas es una biblioteca poderosa para trabajar con datos. En esta actividad, utilizaremos algunas de sus características básicas. La estructura principal que maneja pandas es el "DataFrame", una tabla bidimensional en la que las filas suelen representar casos (en este caso, ciclones tropicales) y las columnas representan variables. Pandas también maneja una estructura unidimensional llamada "Series", que también encontraremos más adelante.

Pandas ofrece varias funciones con el prefijo 'read_' para leer diferentes formatos de archivos. En nuestro caso, nos enfocaremos en la función `read_csv`, que se utiliza para leer archivos de texto delimitados por comas, como los que pueden exportarse desde Excel. De forma predeterminada, `read_csv` asume que la primera fila del archivo contiene los nombres de las columnas, y estos se utilizan para etiquetar las columnas del DataFrame creado.

Usar `read_csv` en su modo predeterminado es sencillo. Sin embargo, existen muchas opciones adicionales que permiten manejar situaciones menos comunes, como cuando el delimitador es un tabulador en lugar de una coma (en ese caso, usaríamos `sep='\t'`). Puedes consultar la documentación completa de `read_csv` para más detalles.

Pandas también permite leer archivos directamente desde internet mediante una URL, como haremos a continuación. En el script de Python, llamaremos al conjunto de datos 'df', que será el nombre de la variable que contendrá el DataFrame después de haber cargado los datos.

El primer argumento de `read_csv` suele ser una URL o el nombre de un archivo en tu computadora. Por ejemplo, la variable `url` en el código de abajo contiene la dirección web donde se encuentran los datos. Si el archivo estuviera en tu sistema local, pasarías la ruta del archivo en lugar de la URL, por ejemplo: `pd.read_csv("mi_archivo.csv")` para leer un archivo llamado `mi_archivo.csv` en el directorio de trabajo actual.

```python
url = "ibtracs.csv"
df = pd.read_csv(url)
```

Para confirmar que hemos obtenido los datos correctos, podemos mostrar la forma (número de filas y columnas) del DataFrame en el notebook. Recuerda que la última expresión en cualquier celda de un Jupyter notebook se imprime automáticamente, pero puedes forzar que otras expresiones se impriman usando la función `print`, por ejemplo, `print(df.shape)`.

Según lo que vemos a continuación, el conjunto de datos cargado contiene X filas, que corresponden a ciclones registrados en IBTrACS, y Y columnas, que corresponden a diferentes variables climáticas y de seguimiento. Aunque IBTrACS recopila muchas más variables, aquí estamos trabajando con un archivo reducido que contiene un subconjunto específico de datos.


In [ ]:
df = pd.read_csv('ibtracs.csv', low_memory = False)

In [ ]:
df.shape

### 2.1 Explorando el contenido de un conjunto de datos

Pandas ofrece varias maneras básicas de entender lo que hay en un conjunto de datos. Por ejemplo, anteriormente usamos el método `shape` para determinar el número de filas y columnas en un conjunto de datos. Las columnas en un DataFrame de Pandas tienen nombres, y para ver esos nombres puedes usar el método `columns`:

```python
df.columns
```


In [ ]:
df.columns

Como ves, son demasiadas columnas y eso es porque IBTRACs contiene información de diferentes fuentes, para fines prácticos te ayudaré a seleccionar solo algunas columnas.

### Descripción de las columnas del conjunto de datos IBTrACS

A continuación, se describen las principales columnas que componen el conjunto de datos de IBTrACS:

- **`SID`**: Identificador único de la tormenta. Este código combina información del ciclón y su temporada para facilitar su rastreo.

- **`SEASON`**: Año en el que se desarrolló la tormenta. Este campo es útil para identificar en qué temporada ciclónica ocurrió el fenómeno.

- **`NUMBER`**: Número de identificación del ciclón dentro de una temporada específica. Por ejemplo, el primer ciclón de la temporada tendrá el número 1.

- **`BASIN`**: Océano o cuenca donde se formó el ciclón (por ejemplo, Atlántico, Pacífico). Ayuda a categorizar los ciclones según la región donde ocurrieron.

- **`SUBBASIN`**: Subcuenca específica dentro de la cuenca principal donde se desarrolló la tormenta (por ejemplo, Golfo de México).

- **`NAME`**: Nombre del ciclón, si se le asignó alguno. En muchos casos, las tormentas más significativas reciben un nombre para ser fácilmente reconocidas.

- **`ISO_TIME`**: Fecha y hora del registro de datos en formato ISO (AAAA-MM-DDTHH:MM:SSZ). Esta columna es esencial para entender la evolución del ciclón en el tiempo.

- **`NATURE`**: Naturaleza del sistema, indicando si es tropical, subtropical o extratropical, entre otros tipos.

- **`LAT`**: Latitud en la que se localiza el ciclón durante la observación. Se expresa en grados decimales.

- **`LON`**: Longitud en la que se localiza el ciclón durante la observación. También se expresa en grados decimales.

- **`WMO_WIND`**: Velocidad máxima del viento (en nudos) según la Organización Meteorológica Mundial (WMO). Este valor es clave para determinar la intensidad de la tormenta.

- **`WMO_PRES`**: Presión central mínima (en hPa) según la WMO. La presión baja es un indicador de la fuerza del ciclón.

- **`WMO_AGENCY`**: Agencia meteorológica que reporta la información para este ciclón (por ejemplo, NOAA, JTWC). Esto es importante para la consistencia de los datos.

- **`TRACK_TYPE`**: Tipo de seguimiento del ciclón, como "operacional" o "reanalizado". Indica la naturaleza de los datos que se están utilizando.

- **`DIST2LAND`**: Distancia a la costa más cercana (en kilómetros). Ayuda a evaluar el potencial de impacto en áreas terrestres.

- **`LANDFALL`**: Indica si el ciclón tocó tierra ("1" para sí, "0" para no). Este dato es clave para evaluar los efectos potenciales de la tormenta.

- **`USA_STATUS`**: Estado del ciclón según los registros de EE.UU., por ejemplo, "tropical storm" o "hurricane".

- **`USA_WIND`**: Velocidad máxima del viento (en nudos) según los registros de EE.UU.

- **`USA_PRES`**: Presión central mínima (en hPa) según los registros de EE.UU.

- **`USA_SSHS`**: Categoría en la escala Saffir-Simpson según EE.UU., que clasifica los ciclones según la intensidad del viento.

- **`USA_R34_NE`**: Radio de vientos de 34 nudos en el cuadrante noreste (en millas náuticas).

- **`USA_R34_SE`**: Radio de vientos de 34 nudos en el cuadrante sureste (en millas náuticas).

- **`USA_R34_SW`**: Radio de vientos de 34 nudos en el cuadrante suroeste (en millas náuticas).

- **`USA_R34_NW`**: Radio de vientos de 34 nudos en el cuadrante noroeste (en millas náuticas).

- **`USA_ROCI`**: Radio de circulación exterior, que indica el alcance general de los vientos (en millas náuticas).

- **`USA_RMW`**: Radio de vientos máximos, indicando la distancia desde el centro de la tormenta hasta donde ocurren los vientos más fuertes (en millas náuticas).

- **`USA_EYE`**: Diámetro del ojo del ciclón, si aplica (en millas náuticas).

- **`STORM_SPEED`**: Velocidad de desplazamiento de la tormenta (en nudos). Indica cuán rápido se mueve el sistema a través del océano o tierra.

- **`STORM_DIR`**: Dirección en grados en la que se desplaza la tormenta (0 a 360 grados).

Este conjunto de datos es esencial para estudiar la trayectoria, intensidad y comportamiento de los ciclones tropicales a nivel mundial.



In [ ]:
columnas = ['SID', 'SEASON', 'NUMBER', 'BASIN', 'SUBBASIN', 'NAME', 'ISO_TIME', 'NATURE', 'LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'WMO_AGENCY', 
'TRACK_TYPE', 'DIST2LAND', 'LANDFALL', 'USA_STATUS', 'USA_WIND', 'USA_PRES', 'USA_SSHS', 'USA_R34_NE', 'USA_R34_SE', 'USA_R34_SW',
 'USA_R34_NW', 'USA_ROCI', 'USA_RMW', 'USA_EYE', 'STORM_SPEED', 'STORM_DIR']

## Aquí hacemos un "slice de la base de datos para solo colocar la información de nuestro interes"

df_slice = df[columnas].copy()

In [ ]:
df_slice.columns


Cada variable en un DataFrame de Pandas tiene un tipo de dato. Hay muchos tipos de datos diferentes, pero los más comunes son valores de punto flotante (números reales), enteros, cadenas (texto) y valores de fecha/hora. Cuando Pandas lee un archivo de texto o CSV, adivina los tipos de datos basándose en lo que ve en las primeras filas del archivo. Usualmente selecciona un tipo adecuado, pero ocasionalmente no lo hace. Para confirmar que los tipos de datos son consistentes con lo que las variables representan, inspecciona el atributo `dtypes` del DataFrame.

In [ ]:
df_slice.dtypes

#### Es hora de algunas preguntas...
1. ¿Con los tipos de datos antes visto podrías decir que tipo de variables son?
2. ¿Consideras que es útil entonces conocer previamente de que tipo de variables hablamos?

## 3. VARIABLES CATEGORICAS

De la lista anterior trabajaremos con algunas variables categóricas expresadas en la columna BASIN. Veamos los valores unicos en esta columna:

In [ ]:
df_slice.BASIN.unique()

Los valores de acuerdo a la documentación deberían ser: 
- NA - North Atlantic
- EP - Eastern North Pacific
- WP - Western North Pacific
- NI - North Indian
- SI - South Indian
- SP - Southern Pacific
- SA - South Atlantic
- MM - Missing - should not appear in final IBTrACS product

Hagamos una tabla de frecuencia para la variable: 

In [ ]:
# Paso 1: Crear la tabla de frecuencia de la columna BASIN
# La función value_counts() cuenta la frecuencia de cada categoría en la columna.
frecuencia = df_slice['BASIN'].value_counts()

In [ ]:
# Paso 2: Convertir la tabla de frecuencia en un DataFrame
# Usamos reset_index() para transformar la serie de frecuencias en un DataFrame.
tabla_frecuencia = frecuencia.reset_index()

In [ ]:
# Paso 3: Renombrar las columnas del DataFrame
# La primera columna será 'BASIN' y la segunda 'FRECUENCIA'.
tabla_frecuencia.columns = ['BASIN', 'FRECUENCIA']

In [ ]:
# Nota: Hasta aquí, la tabla de frecuencia está lista.

# Paso 4: Agregar la columna de frecuencia relativa (porcentaje)
# La frecuencia relativa se obtiene dividiendo cada frecuencia por el total y multiplicando por 100.
tabla_frecuencia['FRECUENCIA_RELATIVA'] = (tabla_frecuencia['FRECUENCIA'] / tabla_frecuencia['FRECUENCIA'].sum()) * 100

In [ ]:
# Paso 5: Agregar la columna de fracción
# La fracción se obtiene dividiendo la frecuencia de cada categoría entre el total.
tabla_frecuencia['FRACCIÓN'] = tabla_frecuencia['FRECUENCIA'] / tabla_frecuencia['FRECUENCIA'].sum()

In [ ]:
# Mostrar la tabla de frecuencias con las nuevas columnas
tabla_frecuencia

In [ ]:
print("\nTotales de cada columna:")
print(f"Total Frecuencia: {tabla_frecuencia['FRECUENCIA'].sum()}")
print(f"Total Frecuencia Relativa (%): {tabla_frecuencia['FRECUENCIA_RELATIVA'].sum()}%")
print(f"Total Fracción: {tabla_frecuencia['FRACCIÓN'].sum()}")

In [ ]:
print("\nConteo de valores nulos en la columna:")
print(df_slice.BASIN.isnull().sum())

In [ ]:
# Paso 6: Contar los valores nulos en la columna BASIN
frecuencia = df_slice['BASIN'].value_counts(dropna = False)
tabla_frecuencia = frecuencia.reset_index()
tabla_frecuencia.columns = ['BASIN', 'FRECUENCIA']
tabla_frecuencia['FRECUENCIA_RELATIVA'] = (tabla_frecuencia['FRECUENCIA'] / tabla_frecuencia['FRECUENCIA'].sum()) * 100
tabla_frecuencia['FRACCIÓN'] = tabla_frecuencia['FRECUENCIA'] / tabla_frecuencia['FRECUENCIA'].sum()
tabla_frecuencia['BASIN'] = tabla_frecuencia['BASIN'].fillna('MM')

In [ ]:
print("\nTotales de cada columna:")
print(f"Total Frecuencia: {tabla_frecuencia['FRECUENCIA'].sum()}")
print(f"Total Frecuencia Relativa (%): {tabla_frecuencia['FRECUENCIA_RELATIVA'].sum()}%")
print(f"Total Fracción: {tabla_frecuencia['FRACCIÓN'].sum()}")

### 3.1 Gráfico de barras por columna y Pareto bar chart. 

In [ ]:
tabla_frecuencia.BASIN.unique()

In [ ]:
# Paso 1: Crear gráfico de barras de la frecuencia
fig1 = plt.figure(figsize=(12, 6))
ax1 = sns.barplot(x='BASIN', y='FRECUENCIA', data=tabla_frecuencia, palette='viridis')
plt.title('Frecuencia de Ciclones por Cuenca')
plt.xlabel('Cuenca')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Crear el gráfico de Pareto
fig2 = plt.figure(figsize=(12, 6))
ax2 =sns.barplot(x='BASIN', y='FRECUENCIA_RELATIVA', data=tabla_frecuencia, palette='viridis')
plt.title('Frecuencia de Ciclones por Cuenca')
plt.xlabel('Cuenca')
plt.ylabel('Frecuencia (%)')
plt.show()



Si te interesa agregar etiquetas: este codigo podría ser util: 

```python
# Agregar etiquetas a las barras
for p in ax1.patches:
    ax1.annotate(format(p.get_height(), '.0f'), 
                 (p.get_x() + p.get_width() / 2., p.get_height()), 
                 ha='center', va='center', 
                 xytext=(0, 9), 
                 textcoords='offset points')              
```

In [ ]:
# Agregar etiquetas a las barras
for p in ax1.patches:
    ax1.annotate(format(p.get_height(), '.0f'), 
                 (p.get_x() + p.get_width() / 2., p.get_height()), 
                 ha='center', va='center', 
                 xytext=(0, 9), 
                 textcoords='offset points')
fig1

In [ ]:
### TU TURNO... AGREGA LAS ETIQUETAS A LA FRECUENCIA RELATIVA




### 3.2 Agregemos un gráfico de pastel

In [ ]:
# Paso 1: Crear gráfico de pastel
plt.figure(figsize=(10, 8))
plt.pie(tabla_frecuencia['FRECUENCIA'], labels=tabla_frecuencia['BASIN'], 
        autopct='%1.1f%%', colors=sns.color_palette('viridis', len(tabla_frecuencia)), 
        startangle=140, wedgeprops={'edgecolor': 'black'})

plt.title('Distribución de Ciclones por Cuenca')
plt.show()

## 4. VARIABLES CUANTITATIVAS

Para ilustrarte lo bello que es Python con variables cuantitativas vamos a trabajar con una serie de columnas que son numéricas. Una numérica discreta ('STORM_SPEED') y otra numérica continua ('USA_WIND).

Ambas variables tienen un carácter de variables discretas veamos:

In [ ]:
print(f"Algunos datos dentro de la variable STORM_SPEED son : \n {df_slice.STORM_SPEED.unique()[:10]} \n")

print(f"Algunos datos dentro de la variable USA_WIND son :\n {df_slice.USA_WIND.unique()[:10]}")

Pero no te preocupes, vamos a cambiar una variable porque las velocidades estan en Nudos, y queremos conocer la velocidad de desplazamiento de los ciclones (STORM_SPEED) en km/h, así que multiplicaremos los valores por 1.854 factor de conversión de un nudo a 1 kmh. 

**CUIDADO** Tenemos un valor vacío en la columna USA_WIND vamos a limpiarlo propiamente. Tenemos varias opciones pero dado que en la velocidad el 0 SI es arbitrario asignaremos al valor ' ' un valor nulo. 

Hagamos las transformaciones propias. 

In [ ]:
df_slice['STORM_SPEED_KMH'] = df_slice.STORM_SPEED.mul(1.854)
df_slice['USA_WIND_CLN'] = df_slice.USA_WIND.replace({' ': np.nan}).astype(float)

In [ ]:
# Definir los intervalos
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120]
labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110', '110-120']

# Categorizar los datos en los intervalos definidos
df_slice['INTERVALO'] = pd.cut(df_slice['STORM_SPEED_KMH'], bins=bins, labels=labels, right=False)

# Contar la frecuencia de cada intervalo
tabla_frecuencia = df_slice['INTERVALO'].value_counts().sort_index().reset_index()
tabla_frecuencia.columns = ['INTERVALO', 'FRECUENCIA']
# tabla_frecuencia['FRECUENCIA_RELATIVA'] = (tabla_frecuencia['FRECUENCIA'] / tabla_frecuencia['FRECUENCIA'].sum()) * 100
tabla_frecuencia

In [ ]:
df_slice['USA_WIND_CLN'].value_counts().reset_index().sort_values(by = 'USA_WIND_CLN')

In [ ]:
# Definir los intervalos
bins = [0, 20, 40, 60, 80, 100, 120, 140, 160, 180]
labels = ['0-20', '20-40', '40-60', '60-80', '80-100', '100-120', '120-140', '140-160', '160-180']

# Categorizar los datos en los intervalos definidos
df_slice['INTERVALO'] = pd.cut(df_slice['USA_WIND_CLN'], bins=bins, labels=labels, right=False)

# Contar la frecuencia de cada intervalo
tabla_frecuencia = df_slice['INTERVALO'].value_counts().sort_index().reset_index()
tabla_frecuencia.columns = ['INTERVALO', 'FRECUENCIA']
# tabla_frecuencia['FRECUENCIA_RELATIVA'] = (tabla_frecuencia['FRECUENCIA'] / tabla_frecuencia['FRECUENCIA'].sum()) * 100
tabla_frecuencia

### 4.1 Medidas de tendencia central

hagamos un SLICE de solo las columnas que usaremos y calculemos las metricas de: MEDIA, MEDIANA Y MODA. 

**Para la variable discreta**

In [ ]:
df_slice.USA_WIND_CLN.mean()

In [ ]:
df_slice.USA_WIND_CLN.median()

¿Que puedes describir de los datos anteriores?



In [ ]:
df_slice.USA_WIND_CLN.mode()

¿Y de la moda?

**Para la variable continua**

In [ ]:
df_slice.STORM_SPEED_KMH.mean()

In [ ]:
df_slice.STORM_SPEED_KMH.median()

¿Que puedes describir de los datos anteriores?

In [ ]:
df_slice.STORM_SPEED_KMH.mode()

___

### EJERCICIO. CONTESTEMOS LAS SIGUIENTES TRES PREGUNTAS: 

A. En un estudio infieren que un ciclón tropical es lento si pertenece al 25% de las velocidades más bajas de los ciclones, y rápidos si superán al limte de los 75% de las velocidades. ¿A qué velocidades nos referimos?

B. Si aplicamos la regla de los eventos extremos a la velocidad del viento del ciclón tropical, ¿Cuál sería dicho umbral?

C. ¿En qué cuenca existen más ciclones extremos?

### 4.2 Medidas de posición. 

In [ ]:
## Pregunta A. Calculemos los cuartiles. El primer cuartil seria para identificar los ciclones lentos y el tercer cuartil 
# sería para definir los ciclones rápidas o lo que es igual los percentiles .25 y .75

df_slice.STORM_SPEED_KMH.quantile([.25, .75])
 

In [ ]:
## Pregunta B. Calculemos el percentil 95 para conocer el extremo de los CTS

df_slice.USA_WIND_CLN.quantile(.95)

In [ ]:
### Pregunta C. En este caso haremos una diferencia entre las cuencas usando la función groupby de pandas y luego calculando aquellos donde el viento sea
# superior a nuestro umbral calculado

df_slice[df_slice['USA_WIND_CLN'] > 108].groupby('BASIN').size().reset_index(name='COUNT_WIND_GT_108')

### 4.3 MEDIDAS DE DISPERSIÓN 
Calculemos ahora las medidas de dispersión, podrías decir si son suficientemente dispersas ...


In [ ]:
columna = 'STORM_SPEED_KMH'

# Calcular el rango
rango = df_slice[columna].max() - df_slice[columna].min()

# Calcular la desviación estándar
desviacion_estandar = df_slice[columna].std()

# Calcular el IQR
Q1 = df_slice[columna].quantile(0.25)
Q3 = df_slice[columna].quantile(0.75)
IQR = Q3 - Q1

# Mostrar los resultados
print(f"Rango: {rango}")
print(f"Desviación Estándar: {desviacion_estandar}")
print(f"IQR: {IQR}")

In [ ]:
columna = 'USA_WIND_CLN'

# Calcular el rango
rango = df_slice[columna].max() - df_slice[columna].min()

# Calcular la desviación estándar
desviacion_estandar = df_slice[columna].std()

# Calcular el IQR
Q1 = df_slice[columna].quantile(0.25)
Q3 = df_slice[columna].quantile(0.75)
IQR = Q3 - Q1

# Mostrar los resultados
print(f"Rango: {rango}")
print(f"Desviación Estándar: {desviacion_estandar}")
print(f"IQR: {IQR}")

## HORA DE COMPROBAR LAS GRAFICAS... ENTRE HISTOGRAMAS Y BOXPLOTS

In [ ]:
## Podemos hacer un histograma simple como los que hemos visto: 
# Configurar el tamaño de la figura
fig, ax = plt.subplots()

# Crear el histograma con seaborn
sns.histplot(df_slice['STORM_SPEED_KMH'], binwidth=10, edgecolor='white', alpha=0.7, kde=False, ax = ax)

# Añadir títulos y etiquetas
plt.title('Histograma de la Velocidad de Tormentas (km/h)')
plt.xlabel('Velocidad de Tormenta (km/h)')
plt.ylabel('Frecuencia')

# Mostrar el gráfico
plt.show()

In [ ]:
mean = np.mean(df_slice['STORM_SPEED_KMH'])
median = np.median(df_slice['STORM_SPEED_KMH'])

In [ ]:
# Agregar líneas para la media y la mediana
ax.axvline(mean, color='red', linestyle='dashed', linewidth=1)
ax.axvline(median, color='blue', linestyle='dashed', linewidth=1)

# Añadir una leyenda
display(fig)


In [ ]:
## GRAFICO PARA VELOCIDAD DEL VIENTO

fig, ax = plt.subplots()

# Crear el histograma con seaborn
sns.histplot(df_slice['USA_WIND_CLN'], binwidth=20, edgecolor='white', alpha=0.7, kde=False, ax = ax)
mean = np.mean(df_slice['USA_WIND_CLN'])
median = np.median(df_slice['USA_WIND_CLN'].dropna())
print(mean)
print(median)
ax.axvline(mean, color='red', linestyle='dashed', linewidth=1)
ax.axvline(median, color='blue', linestyle='dashed', linewidth=1)
# Añadir títulos y etiquetas
plt.title('Histograma de la Velocidad de Tormentas (km/h)')
plt.xlabel('Velocidad de Tormenta (km/h)')
plt.ylabel('Frecuencia')

# Mostrar el gráfico
plt.show()

¿Cómo se comportan las variables?

¿Qué información hace falta en estos gráficos?

## vamos con las cajas y bigotes...

¿Sabrías decirme en que cuenca los ciclones son más intensos en vientos (No necesariamente por el p95)?

¿Y en que cuenca hay ciclones más rápidos e intensos?

In [ ]:
df_slice

In [ ]:
# Primera pregunta
plt.figure(figsize=(10, 6))
sns.boxplot(y='USA_WIND_CLN', x = 'BASIN', hue='BASIN', data=df_slice, palette='Set1',fill=False, gap=.1, legend =False)
plt.yticks(ticks=range(10,171,20))


# Etiquetas y título
plt.title('Boxplot de Velocidades Intensivas del Viento')


# Mostrar el boxplot
plt.show()

In [ ]:
## Para la pregunta B 

# Calcular los cuartiles de 'STORM_SPEED_KMH'
q1 = df_slice['STORM_SPEED_KMH'].quantile(0.25)
q3 = df_slice['STORM_SPEED_KMH'].quantile(0.75)

# Crear la nueva columna de clasificación
def classify_storm_speed(speed):
    if speed <= q1:
        return 'Ciclón Lento'
    elif speed >= q3:
        return 'Ciclón Rápido'
    else:
        return 'Ciclón Normal'

df_slice['STORM_CLASS'] = df_slice['STORM_SPEED_KMH'].apply(classify_storm_speed)

In [ ]:
# Crear el boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(y = 'USA_WIND_CLN', x='BASIN', hue='STORM_CLASS', data=df_slice, palette='Set1')

# Etiquetas y título
plt.title('Boxplot de Velocidades del Viento por Cuenca y Tipo de Ciclón')
plt.xlabel('Velocidades del Viento (m/s)')
plt.ylabel('Tipo de Ciclón')

# Mostrar el boxplot
plt.show()

¿En que cuenca hay ciclones más lentos?

¿Que sean ciclones lentos implica tengan más o menor intensidad de sus vientos?